In [1]:
%pwd

'd:\\ML_Deployment\\Automadata\\research'

In [2]:
import os 
os.chdir('../')

In [3]:
%pwd

'd:\\ML_Deployment\\Automadata'

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass
class ModelEvaluationConfig:
    root_dir: Path 
    test_data_path: Path
    model_path: Path
    metric_file_name: Path

In [5]:
from src.constants import *
from src.utils.common import *

In [6]:
import pandas as pd 
import numpy as np 
import joblib
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, f1_score

In [7]:
class ConfigManager:
    def __init__(self,
            config_filepath = CONFIG_FILE_PATH):
        self.config = read_yaml(config_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self)-> ModelEvaluationConfig:
        config = self.config.model_evaluation 
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path =config.test_data_path,
            model_path=config.model_path,
            metric_file_name = config.metric_file_name
        )
        
        return model_evaluation_config

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        f1 = f1_score(actual, pred)
        roc_auc = roc_auc_score(actual, pred)

        return accuracy, precision, recall, f1, roc_auc

    def save_eval_metrics(self):
        test_data = np.load(self.config.test_data_path)
        X_test = test_data[:, :-1]
        y_test = test_data[:, -1]

        model = joblib.load(self.config.model_path)
        y_pred = model.predict(X_test)

        accuracy, precision, recall, f1, roc_auc = self.eval_metrics(actual=y_test, pred=y_pred)

        scores = {
            "accuracy_score": round(accuracy, 4),
            "precision_score": round(precision, 4),
            "recall_score": round(recall, 4),
            "f1_score": round(f1, 4),
            "roc_auc_score": round(roc_auc, 4)
        }
        save_json(path=Path(self.config.metric_file_name), data=scores)


In [11]:
try:
    config = ConfigManager()
    model_eval_config = config.get_model_evaluation_config()
    model_eval_config = ModelEvaluation(config=model_eval_config)
    model_eval_config.save_eval_metrics()
except Exception as e:
    raise e 

[2024-01-15 17:46:08,874: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-15 17:46:08,874: INFO: common: created directory at: artifacts]
[2024-01-15 17:46:08,882: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-15 17:46:09,817: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
